# ENTREGABLE 4

# INSTRUCCIONES

Utilizar el archivo CSV (`dataset_banco_clean.csv`) con 45189 filas y 17 columnas y aplicar las técnicas de normalización del entregable 3.

In [1]:
# imports
from sklearn.datasets import load_iris
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import VotingClassifier


In [75]:
import warnings
from sklearn.exceptions import ConvergenceWarning

# Ignorar los ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)

In [2]:
ruta = "./dataset_banco_clean.csv"
data = pd.read_csv(ruta)
data

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143.0,yes,no,unknown,5,may,261.0,1,-1.0,0,unknown,no
1,44,technician,single,secondary,no,29.0,yes,no,unknown,5,may,151.0,1,-1.0,0,unknown,no
2,33,entrepreneur,married,secondary,no,2.0,yes,yes,unknown,5,may,76.0,1,-1.0,0,unknown,no
3,47,blue-collar,married,unknown,no,1506.0,yes,no,unknown,5,may,92.0,1,-1.0,0,unknown,no
4,33,unknown,single,unknown,no,1.0,no,no,unknown,5,may,198.0,1,-1.0,0,unknown,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45184,51,technician,married,tertiary,no,825.0,no,no,cellular,17,nov,977.0,3,-1.0,0,unknown,yes
45185,71,retired,divorced,primary,no,1729.0,no,no,cellular,17,nov,456.0,2,-1.0,0,unknown,yes
45186,72,retired,married,secondary,no,5715.0,no,no,cellular,17,nov,1127.0,5,184.0,3,success,yes
45187,57,blue-collar,married,secondary,no,668.0,no,no,telephone,17,nov,508.0,4,-1.0,0,unknown,no


# Objetivo

Generar un modelo de clasificación capaz de predecir si los clientes están interesados o no en adquirir un certificado de depósito a término con el banco en función de las carácterísticas del dataset

* Aplicar las técnicas oportunas de procesamiento de datos

* Generar split de los datos

* Valorar diferentes modelos de clasificación

* Comparación entre modelos

* Ensemble

* Métricas

* Conclusiones finales

# 1. PROCESAMIENTO

El primer paso que vamos a realizar antes de dividir los sets de datos para empezar a valorar diferentes modelos es convertir las variables categóricas. De este forma conseguiremos que cada categoría sea representada por un número en lugar de una cadena de texto.

Por ejemplo, las observaciones de las variables que tengan respuestas de "sí" o "no" serán representadas por el valor 0 si la respuesta es "no" y por el valor 1 si la respuesta es "sí".

In [3]:
data.head(5)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143.0,yes,no,unknown,5,may,261.0,1,-1.0,0,unknown,no
1,44,technician,single,secondary,no,29.0,yes,no,unknown,5,may,151.0,1,-1.0,0,unknown,no
2,33,entrepreneur,married,secondary,no,2.0,yes,yes,unknown,5,may,76.0,1,-1.0,0,unknown,no
3,47,blue-collar,married,unknown,no,1506.0,yes,no,unknown,5,may,92.0,1,-1.0,0,unknown,no
4,33,unknown,single,unknown,no,1.0,no,no,unknown,5,may,198.0,1,-1.0,0,unknown,no


In [4]:
data['housing'].value_counts()

housing
yes    25111
no     20078
Name: count, dtype: int64

In [5]:
data['default'].value_counts()

default
no     44374
yes      815
Name: count, dtype: int64

In [6]:
data.replace({'no': 0, 'yes': 1}, inplace = True)

In [7]:
data['housing'].value_counts()

housing
1    25111
0    20078
Name: count, dtype: int64

In [8]:
data['default'].value_counts()

default
0    44374
1      815
Name: count, dtype: int64

In [9]:
data['month'].value_counts()

month
may    13748
jul     6895
aug     6246
jun     5341
nov     3970
apr     2931
feb     2648
jan     1402
oct      738
sep      579
mar      477
dec      214
Name: count, dtype: int64

In [10]:
data['month'].replace({'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4, 'may': 5, 'jun': 6, 'jul': 7, 'aug': 8, 'sep': 9, 'oct': 10, 'nov': 11, 'dec': 12}, inplace=True)

In [11]:
data['month'].value_counts()

month
5     13748
7      6895
8      6246
6      5341
11     3970
4      2931
2      2648
1      1402
10      738
9       579
3       477
12      214
Name: count, dtype: int64

In [12]:
data['education'].replace({'primary': 1, 'secondary': 2, 'tertiary': 3, 'unknown': 0}, inplace = True)

In [13]:
data

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,3,0,2143.0,1,0,unknown,5,5,261.0,1,-1.0,0,unknown,0
1,44,technician,single,2,0,29.0,1,0,unknown,5,5,151.0,1,-1.0,0,unknown,0
2,33,entrepreneur,married,2,0,2.0,1,1,unknown,5,5,76.0,1,-1.0,0,unknown,0
3,47,blue-collar,married,0,0,1506.0,1,0,unknown,5,5,92.0,1,-1.0,0,unknown,0
4,33,unknown,single,0,0,1.0,0,0,unknown,5,5,198.0,1,-1.0,0,unknown,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45184,51,technician,married,3,0,825.0,0,0,cellular,17,11,977.0,3,-1.0,0,unknown,1
45185,71,retired,divorced,1,0,1729.0,0,0,cellular,17,11,456.0,2,-1.0,0,unknown,1
45186,72,retired,married,2,0,5715.0,0,0,cellular,17,11,1127.0,5,184.0,3,success,1
45187,57,blue-collar,married,2,0,668.0,0,0,telephone,17,11,508.0,4,-1.0,0,unknown,0


In [14]:
columnas_categoricas = ['job', 'marital', 'contact', 'poutcome']
    
category_mapping = {}
for column in columnas_categoricas:
    # Obtiene las categorías únicas en la columna
    categories = data[column].unique()
    # Crea un diccionario para mapear cada categoría a su valor numérico correspondiente
    category_mapping[column] = {category: i for i, category in enumerate(categories)}
    # Etiqueta numéricamente las categorías en la columna
    data[column] = data[column].map(category_mapping[column])


print("\nMapeo de categorías a valores numéricos:")
category_mapping


Mapeo de categorías a valores numéricos:


{'job': {'management': 0,
  'technician': 1,
  'entrepreneur': 2,
  'blue-collar': 3,
  'unknown': 4,
  'retired': 5,
  'administrative': 6,
  'services': 7,
  'self-employed': 8,
  'unemployed': 9,
  'housemaid': 10,
  'student': 11},
 'marital': {'married': 0, 'single': 1, 'divorced': 2},
 'contact': {'unknown': 0, 'cellular': 1, 'telephone': 2},
 'poutcome': {'unknown': 0, 'failure': 1, 'other': 2, 'success': 3}}

In [15]:
data

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,0,0,3,0,2143.0,1,0,0,5,5,261.0,1,-1.0,0,0,0
1,44,1,1,2,0,29.0,1,0,0,5,5,151.0,1,-1.0,0,0,0
2,33,2,0,2,0,2.0,1,1,0,5,5,76.0,1,-1.0,0,0,0
3,47,3,0,0,0,1506.0,1,0,0,5,5,92.0,1,-1.0,0,0,0
4,33,4,1,0,0,1.0,0,0,0,5,5,198.0,1,-1.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45184,51,1,0,3,0,825.0,0,0,1,17,11,977.0,3,-1.0,0,0,1
45185,71,5,2,1,0,1729.0,0,0,1,17,11,456.0,2,-1.0,0,0,1
45186,72,5,0,2,0,5715.0,0,0,1,17,11,1127.0,5,184.0,3,3,1
45187,57,3,0,2,0,668.0,0,0,2,17,11,508.0,4,-1.0,0,0,0


# 2. SEPARAR SETS

In [16]:
X = data.drop(columns = 'y')
y = data['y']

'''Crear los sets de entrenamiento (70%), validación (15%) y prueba (15%)'''

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state= 47)

x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=15/30, random_state=47)

# Verificar
print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)
print(x_test.shape, y_test.shape)

(31632, 16) (31632,)
(6778, 16) (6778,)
(6779, 16) (6779,)


# 3. NORMALIZAR

In [22]:
# Haremos escalamiento entre 0 y 1 (normalización)

def escalar_datos(xtr, xvl, xts):
    '''Escalar datos de entrada (entrenamiento, validación y prueba)
    en el rango de 0 a 1'''
    scaler = MinMaxScaler(feature_range = (0,1))

    # Fit + transform sobre el set de entrenamiento
    xtr_s = scaler.fit_transform(xtr)

    # Transform sobre prueba y validación
    xvl_s = scaler.transform(xvl)
    xts_s = scaler.transform(xts)

    return xtr_s, xvl_s, xts_s

# Ejecutar la función
x_train_s, x_val_s, x_test_s = escalar_datos(x_train, x_val, x_test)


# 4. MODELOS Y MÉTRICAS

In [27]:
accuracy_validation = {}
accuracy_test = {}
f1_validation = {}
f1_test = {}

## 4.1. Árbol de Decisión

In [28]:
dt = DecisionTreeClassifier()

# Entrenar el clasificador
dt.fit(x_train_s, y_train)

# Predecir en el conjunto de VALIDACIÓN
y_pred_val_dt= dt.predict(x_val_s)

# Métricas validación
val_accuracy_dt = accuracy_score(y_val, y_pred_val_dt)
val_f1_dt = f1_score(y_val, y_pred_val_dt)
matrix_val_dt = confusion_matrix(y_val, y_pred_val_dt)

accuracy_validation['DecisionTree'] = val_accuracy_dt
f1_validation['DecisionTree'] = val_f1_dt

print("Precisión del clasificador de árbol de decisión en validación:", val_accuracy_dt)
print("F1-Score del clasificador de árbol de decisión en validación:", val_f1_dt)
print("Matriz de confusión del árbol de decisión en validación:\n" , matrix_val_dt)

Precisión del clasificador de árbol de decisión en validación: 0.8769548539392151
F1-Score del clasificador de árbol de decisión en validación: 0.4920828258221681
Matriz de confusión del árbol de decisión en validación:
 [[5540  423]
 [ 411  404]]


In [43]:
# Predecir en el conjunto de PRUEBA
y_pred_test_dt= dt.predict(x_test_s)

# Métricas PRUEBA
test_accuracy_dt = accuracy_score(y_test, y_pred_test_dt)
test_f1_dt = f1_score(y_test, y_pred_test_dt)
matrix_test_dt = confusion_matrix(y_test, y_pred_test_dt)

accuracy_test['DecisionTree'] = test_accuracy_dt
f1_test['DecisionTree'] = test_f1_dt

print("Precisión del clasificador de árbol de decisión en prueba:", test_accuracy_dt)
print("F1-Score del clasificador de árbol de decisión en prueba:", test_f1_dt)
print("Matriz de confusión del árbol de decisión en prueba:\n", matrix_test_dt)

Precisión del clasificador de árbol de decisión en prueba: 0.8802183212863254
F1-Score del clasificador de árbol de decisión en prueba: 0.4781491002570694
Matriz de confusión del árbol de decisión en prueba:
 [[5595  420]
 [ 392  372]]


## 4.2. KNN

In [32]:
# Grid Search
grid_knn = {
    'n_neighbors': [3, 5, 7, 9, 11], 
    'weights': ['uniform', 'distance']
}

# Inicializar el modelo SVM
modelo_knn = KNeighborsClassifier()

# Realizar la búsqueda de hiperparámetros utilizando RandomSearchCV
grid_search_knn = GridSearchCV(modelo_knn, grid_knn)
grid_search_knn.fit(x_train_s, y_train)

# Obtener los mejores hiperparámetros después de la búsqueda
mejores_hiperparametros_knn = grid_search_knn.best_params_
print("Mejores hiperparámetros encontrados:", mejores_hiperparametros_knn)

Mejores hiperparámetros encontrados: {'n_neighbors': 11, 'weights': 'uniform'}


In [33]:
knn = KNeighborsClassifier(n_neighbors=11, weights='uniform')

# Entrenar el clasificador
knn.fit(x_train_s, y_train)

# Predecir en el conjunto de VALIDACIÓN
y_pred_val_knn= knn.predict(x_val_s)

# Métricas validación
val_accuracy_knn = accuracy_score(y_val, y_pred_val_knn)
val_f1_knn = f1_score(y_val, y_pred_val_knn)
matrix_val_knn = confusion_matrix(y_val, y_pred_val_knn)

accuracy_validation['KNeighbors'] = val_accuracy_knn
f1_validation['KNeighbors'] = val_f1_knn

print("Precisión del clasificador de KNN en validación:", val_accuracy_knn)
print("F1-Score del clasificador de KNN en validación:", val_f1_knn)
print("Matriz de confusión del KNN en validación:\n" , matrix_val_knn)

Precisión del clasificador de KNN en validación: 0.8908232516966657
F1-Score del clasificador de KNN en validación: 0.3211009174311927
Matriz de confusión del KNN en validación:
 [[5863  100]
 [ 640  175]]


In [44]:
# Predecir en el conjunto de PRUEBA
y_pred_test_knn= knn.predict(x_test_s)

# Métricas PRUEBA
test_accuracy_knn = accuracy_score(y_test, y_pred_test_knn)
test_f1_knn = f1_score(y_test, y_pred_test_knn)
matrix_test_knn = confusion_matrix(y_test, y_pred_test_knn)

accuracy_test['KNeighbors'] = test_accuracy_knn
f1_test['KNeighbors'] = test_f1_knn

print("Precisión del clasificador de KNN en prueba:", test_accuracy_knn)
print("F1-Score del clasificador de KNN en prueba:", test_f1_knn)
print("Matriz de confusión del KNN en prueba:\n", matrix_test_knn)

Precisión del clasificador de KNN en prueba: 0.8962973889954271
F1-Score del clasificador de KNN en prueba: 0.3259827420901246
Matriz de confusión del KNN en prueba:
 [[5906  109]
 [ 594  170]]


## 4.3. SVM

In [38]:
# GridSearch
grid_svm = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf']
}

# Inicializar el modelo SVM
svm = SVC()

# Realizar la búsqueda de hiperparámetros utilizando RandomSearchCV
grid_search_svm = GridSearchCV(svm, grid_svm)
grid_search_svm.fit(x_train_s, y_train)

# Obtener los mejores hiperparámetros después de la búsqueda
mejores_hiperparametros_svm = grid_search_svm.best_params_
print("Mejores hiperparámetros encontrados:", mejores_hiperparametros_svm)

Mejores hiperparámetros encontrados: {'C': 10, 'kernel': 'rbf'}


In [40]:
svm = SVC(C=10, kernel='rbf')

# Entrenar el clasificador
svm.fit(x_train_s, y_train)

# Predecir en el conjunto de VALIDACIÓN
y_pred_val_svm= svm.predict(x_val_s)

# Métricas validación
val_accuracy_svm = accuracy_score(y_val, y_pred_val_svm)
val_f1_svm = f1_score(y_val, y_pred_val_svm)
matrix_val_svm = confusion_matrix(y_val, y_pred_val_svm)

accuracy_validation['SVM'] = val_accuracy_svm
f1_validation['SVM'] = val_f1_svm

print("Precisión del clasificador SVM en validación:", val_accuracy_svm)
print("F1-Score del clasificador SVM en validación:", val_f1_svm)
print("Matriz de confusión del SVM en validación:\n" , matrix_val_svm)

Precisión del clasificador SVM en validación: 0.893626438477427
F1-Score del clasificador SVM en validación: 0.34751131221719456
Matriz de confusión del SVM en validación:
 [[5865   98]
 [ 623  192]]


In [45]:
# Predecir en el conjunto de PRUEBA
y_pred_test_svm= svm.predict(x_test_s)

# Métricas PRUEBA
test_accuracy_svm = accuracy_score(y_test, y_pred_test_svm)
test_f1_svm = f1_score(y_test, y_pred_test_svm)
matrix_test_svm = confusion_matrix(y_test, y_pred_test_svm)

accuracy_test['SVM'] = test_accuracy_svm
f1_test['SVM'] = test_f1_svm

print("Precisión del clasificador SVM en prueba:", test_accuracy_svm)
print("F1-Score del clasificador SVM en prueba:", test_f1_svm)
print("Matriz de confusión del SVM en prueba:\n", matrix_test_svm)

Precisión del clasificador SVM en prueba: 0.9016079067709102
F1-Score del clasificador SVM en prueba: 0.3604985618408437
Matriz de confusión del SVM en prueba:
 [[5924   91]
 [ 576  188]]


## 4.4 XGBoost

In [42]:
#GridSearch
grid_xgb = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.1, 0.01, 0.001],
    'n_estimators': [100, 200, 300],
    'gamma': [0, 0.1, 0.2]
}

# Crear un clasificador XGBoost
xgboost = xgb.XGBClassifier()

# Crear un objeto GridSearchCV
grid_search_xgb = GridSearchCV(estimator=xgboost, param_grid=grid_xgb, cv=3, scoring='accuracy', n_jobs=-1)

# Ajustar el objeto GridSearchCV a los datos de entrenamiento
grid_search_xgb.fit(x_train_s, y_train)

# Obtener los mejores parámetros encontrados
best_params_xgb = grid_search_xgb.best_params_
print("Mejores parámetros:", best_params_xgb)

Mejores parámetros: {'gamma': 0.2, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 200}


In [46]:
xgboost = xgb.XGBClassifier(max_depth = 5, learning_rate = 0.1, n_estimators = 200, gamma = 0.2)

# Entrenar el clasificador
xgboost.fit(x_train_s, y_train)

# Predecir en el conjunto de VALIDACIÓN
y_pred_val_xgb= xgboost.predict(x_val_s)

# Métricas validación
val_accuracy_xgb = accuracy_score(y_val, y_pred_val_xgb)
val_f1_xgb = f1_score(y_val, y_pred_val_xgb)
matrix_val_xgb = confusion_matrix(y_val, y_pred_val_xgb)

accuracy_validation['XGBoost'] = val_accuracy_xgb
f1_validation['XGBoost'] = val_f1_xgb

print("Precisión del clasificador XGBoost en validación:", val_accuracy_xgb)
print("F1-Score del clasificador XGBoost en validación:", val_f1_xgb)
print("Matriz de confusión del XGBoost en validación:\n" , matrix_val_xgb)

Precisión del clasificador XGBoost en validación: 0.908084980820301
F1-Score del clasificador XGBoost en validación: 0.5475671750181554
Matriz de confusión del XGBoost en validación:
 [[5778  185]
 [ 438  377]]


In [47]:
# Predecir en el conjunto de PRUEBA
y_pred_test_xgb= xgboost.predict(x_test_s)

# Métricas PRUEBA
test_accuracy_xgb = accuracy_score(y_test, y_pred_test_xgb)
test_f1_xgb = f1_score(y_test, y_pred_test_xgb)
matrix_test_xgb = confusion_matrix(y_test, y_pred_test_xgb)

accuracy_test['XGBoost'] = test_accuracy_xgb
f1_test['XGBoost'] = test_f1_xgb

print("Precisión del clasificador XGBoost en prueba:", test_accuracy_xgb)
print("F1-Score del clasificador XGBoost en prueba:", test_f1_xgb)
print("Matriz de confusión del XGBoost en prueba:\n", matrix_test_xgb)

Precisión del clasificador XGBoost en prueba: 0.9097211978167872
F1-Score del clasificador XGBoost en prueba: 0.5349544072948328
Matriz de confusión del XGBoost en prueba:
 [[5815  200]
 [ 412  352]]


## 4.5. Random Forest

In [49]:
#GridSearch
grid_rf = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30]
}

# Crear un clasificador XGBoost
rf = RandomForestClassifier()

# Crear un objeto GridSearchCV
grid_search_rf = GridSearchCV(rf, grid_rf, cv=5, scoring='accuracy')

# Ajustar el objeto GridSearchCV a los datos de entrenamiento
grid_search_rf.fit(x_train_s, y_train)

# Obtener los mejores parámetros encontrados
best_params_rf = grid_search_rf.best_params_
print("Mejores parámetros:", best_params_rf)

Mejores parámetros: {'max_depth': None, 'n_estimators': 200}


In [50]:
rf = RandomForestClassifier(n_estimators = 200, max_depth = None)

# Entrenar el clasificador
rf.fit(x_train_s, y_train)

# Predecir en el conjunto de VALIDACIÓN
y_pred_val_rf= rf.predict(x_val_s)

# Métricas validación
val_accuracy_rf = accuracy_score(y_val, y_pred_val_rf)
val_f1_rf = f1_score(y_val, y_pred_val_rf)
matrix_val_rf = confusion_matrix(y_val, y_pred_val_rf)

accuracy_validation['Random Forest'] = val_accuracy_rf
f1_validation['Random Forest'] = val_f1_rf

print("Precisión del clasificador Random Forest en validación:", val_accuracy_rf)
print("F1-Score del clasificador Random Forest en validación:", val_f1_rf)
print("Matriz de confusión del Random Forest en validación:\n" , matrix_val_rf)

Precisión del clasificador Random Forest en validación: 0.903953968722337
F1-Score del clasificador Random Forest en validación: 0.5086792452830189
Matriz de confusión del Random Forest en validación:
 [[5790  173]
 [ 478  337]]


In [51]:
# Predecir en el conjunto de PRUEBA
y_pred_test_rf= rf.predict(x_test_s)

# Métricas PRUEBA
test_accuracy_rf = accuracy_score(y_test, y_pred_test_rf)
test_f1_rf = f1_score(y_test, y_pred_test_rf)
matrix_test_rf = confusion_matrix(y_test, y_pred_test_rf)

accuracy_test['Random Forest'] = test_accuracy_rf
f1_test['Random Forest'] = test_f1_rf

print("Precisión del clasificador Random Forest en prueba:", test_accuracy_rf)
print("F1-Score del clasificador Random Forest en prueba:", test_f1_rf)
print("Matriz de confusión del Random Forest en prueba:\n", matrix_test_rf)

Precisión del clasificador Random Forest en prueba: 0.9066233957810886
F1-Score del clasificador Random Forest en prueba: 0.498812351543943
Matriz de confusión del Random Forest en prueba:
 [[5831  184]
 [ 449  315]]


## 4.6. Logistic Regression

In [52]:
#GridSearch
grid_lr = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear', 'saga']  
}

# Crear un clasificador de Regresión Logística
lr = LogisticRegression()

# Crear un objeto GridSearchCV
grid_search_lr = GridSearchCV(lr, grid_lr, cv=5, scoring='accuracy')

# Ajustar el objeto GridSearchCV a los datos de entrenamiento
grid_search_lr.fit(x_train_s, y_train)

# Obtener los mejores parámetros encontrados
best_params_lr = grid_search_lr.best_params_
print("Mejores parámetros:", best_params_lr)

Mejores parámetros: {'C': 0.1, 'penalty': 'l1', 'solver': 'saga'}


In [53]:
lr = LogisticRegression(C = 0.1, penalty = 'l1', solver = 'saga')

# Entrenar el clasificador
lr.fit(x_train_s, y_train)

# Predecir en el conjunto de VALIDACIÓN
y_pred_val_lr = lr.predict(x_val_s)

# Métricas de validación
val_accuracy_lr = accuracy_score(y_val, y_pred_val_lr)
val_f1_lr = f1_score(y_val, y_pred_val_lr)
matrix_val_lr= confusion_matrix(y_val, y_pred_val_lr)

accuracy_validation['Logistic Regression'] = val_accuracy_lr
f1_validation['Logistic Regression'] = val_f1_lr

print("Precisión del clasificador de Regresión Logística en validación:", val_accuracy_lr)
print("F1-Score del clasificador de Regresión Logística en validación:", val_f1_lr)
print("Matriz de confusión de la Regresión Logística en validación:\n", matrix_val_lr)


Precisión del clasificador de Regresión Logística en validación: 0.8971673059899675
F1-Score del clasificador de Regresión Logística en validación: 0.3975799481417459
Matriz de confusión de la Regresión Logística en validación:
 [[5851  112]
 [ 585  230]]


In [54]:
# Predecir en el conjunto de PRUEBA
y_pred_test_lr= lr.predict(x_test_s)

# Métricas PRUEBA
test_accuracy_lr = accuracy_score(y_test, y_pred_test_lr)
test_f1_lr = f1_score(y_test, y_pred_test_lr)
matrix_test_lr = confusion_matrix(y_test, y_pred_test_lr)

accuracy_test['Logistic Regression'] = test_accuracy_lr
f1_test['Logistic Regression'] = test_f1_lr

print("Precisión del clasificador de Regresión Logística en prueba:", test_accuracy_lr)
print("F1-Score del clasificador de Regresión Logística en prueba:", test_f1_lr)
print("Matriz de confusión del de Regresión Logística en prueba:\n", matrix_test_lr)

Precisión del clasificador de Regresión Logística en prueba: 0.9014603923882578
F1-Score del clasificador de Regresión Logística en prueba: 0.3938294010889292
Matriz de confusión del de Regresión Logística en prueba:
 [[5894  121]
 [ 547  217]]


## 4.7. Naive Bayes

In [55]:
nb = GaussianNB()

# Entrenar el clasificador
nb.fit(x_train_s, y_train)

# Predecir en el conjunto de VALIDACIÓN
y_pred_val_nb = nb.predict(x_val_s)

# Métricas de validación
val_accuracy_nb = accuracy_score(y_val, y_pred_val_nb)
val_f1_nb = f1_score(y_val, y_pred_val_nb)
matrix_val_nb= confusion_matrix(y_val, y_pred_val_nb)

accuracy_validation['Naive Bayes'] = val_accuracy_nb
f1_validation['Naive Bayes'] = val_f1_nb

print("Precisión del clasificador Naive Bayes en validación:", val_accuracy_nb)
print("F1-Score del clasificador Naive Bayes en validación:", val_f1_nb)
print("Matriz de confusión de Naive Bayes en validación:\n", matrix_val_nb)

Precisión del clasificador Naive Bayes en validación: 0.8400708173502508
F1-Score del clasificador Naive Bayes en validación: 0.42462845010615713
Matriz de confusión de Naive Bayes en validación:
 [[5294  669]
 [ 415  400]]


In [56]:
# Predecir en el conjunto de PRUEBA
y_pred_test_nb= nb.predict(x_test_s)

# Métricas PRUEBA
test_accuracy_nb = accuracy_score(y_test, y_pred_test_nb)
test_f1_nb = f1_score(y_test, y_pred_test_nb)
matrix_test_nb = confusion_matrix(y_test, y_pred_test_nb)

accuracy_test['Naive Bayes'] = test_accuracy_nb
f1_test['Naive Bayes'] = test_f1_nb

print("Precisión del clasificador Naive Bayes en prueba:", test_accuracy_nb)
print("F1-Score del clasificador Naive Bayes en prueba:", test_f1_nb)
print("Matriz de confusión de Naive Bayes en prueba:\n", matrix_test_nb)

Precisión del clasificador Naive Bayes en prueba: 0.840831981118159
F1-Score del clasificador Naive Bayes en prueba: 0.41644131963223363
Matriz de confusión de Naive Bayes en prueba:
 [[5315  700]
 [ 379  385]]


# 5. COMPARACIÓN ENTRE MODELOS

In [61]:
accuracy_validation, accuracy_test

({'DecisionTree': 0.8769548539392151,
  'KNeighbors': 0.8908232516966657,
  'SVM': 0.893626438477427,
  'XGBoost': 0.908084980820301,
  'Random Forest': 0.903953968722337,
  'Logistic Regression': 0.8971673059899675,
  'Naive Bayes': 0.8400708173502508},
 {'DecisionTree': 0.8802183212863254,
  'KNeighbors': 0.8962973889954271,
  'SVM': 0.9016079067709102,
  'XGBoost': 0.9097211978167872,
  'Random Forest': 0.9066233957810886,
  'Logistic Regression': 0.9014603923882578,
  'Naive Bayes': 0.840831981118159})

In [65]:
# Convertir diccionarios a DataFrames
df1 = pd.DataFrame.from_dict(accuracy_validation, orient='index', columns=['Validation Accuracy'])
df2 = pd.DataFrame.from_dict(accuracy_test, orient='index', columns=['Test Accuracy'])

# Fusionar DataFrames
accuracy_df = pd.merge(df1, df2, left_index=True, right_index=True)

# Mostrar DataFrame
print(accuracy_df)


                     Validation Accuracy  Test Accuracy
DecisionTree                    0.876955       0.880218
KNeighbors                      0.890823       0.896297
SVM                             0.893626       0.901608
XGBoost                         0.908085       0.909721
Random Forest                   0.903954       0.906623
Logistic Regression             0.897167       0.901460
Naive Bayes                     0.840071       0.840832


In [67]:
df3 = pd.DataFrame.from_dict(f1_validation, orient='index', columns=['Validation F1-Score'])
df4 = pd.DataFrame.from_dict(f1_test, orient='index', columns=['Test F1-Score'])

f1score_df = pd.merge(df3, df4, left_index=True, right_index=True)

print(f1score_df)

                     Validation F1-Score  Test F1-Score
DecisionTree                    0.492083       0.478149
KNeighbors                      0.321101       0.325983
SVM                             0.347511       0.360499
XGBoost                         0.547567       0.534954
Random Forest                   0.508679       0.498812
Logistic Regression             0.397580       0.393829
Naive Bayes                     0.424628       0.416441


# 6. ENSEMBLE

In [77]:
clf1 = DecisionTreeClassifier()
clf2 = KNeighborsClassifier(n_neighbors=11, weights='uniform')
clf3 = SVC(C=10, kernel='rbf')
clf4 = xgb.XGBClassifier(max_depth = 5, learning_rate = 0.1, n_estimators = 200, gamma = 0.2)
clf5 = RandomForestClassifier(n_estimators = 200, max_depth = None)
clf6 = LogisticRegression(C = 0.1, penalty = 'l1', solver = 'saga', max_iter=1000)
clf7 = GaussianNB()

ensemble = VotingClassifier(
    estimators=[
        ('DecisionTree', clf1),
        ('KNeighbors', clf2),
        ('SVC', clf3),
        ('XGBoost', clf4),
        ('RandomForest', clf5),
        ('LogisticRegression', clf6),
        ('GaussianNB', clf7)
    ],
    voting='hard'  # 'hard' para votación mayoritaria
)

# Entrenar el ensemble
ensemble.fit(x_train_s, y_train)

# Predecir set de validación
y_pred_val = ensemble.predict(x_val_s)

# Evaluar en validación
accuracy_val = accuracy_score(y_val, y_pred_val)
f1_val = f1_score(y_val, y_pred_val)
confusion_matrix_val = confusion_matrix(y_val, y_pred_val)

print("Accuracy:", accuracy_val)
print("F1-score:", f1_val)
print("Confusion matrix:\n", confusion_matrix_val)

Accuracy: 0.9014458542342874
F1-score: 0.451559934318555
Confusion matrix:
 [[5835  128]
 [ 540  275]]


In [78]:
# Predecir set de prueba
y_pred_test = ensemble.predict(x_test_s)

# Evaluar en prueba
accuracy_test = accuracy_score(y_test, y_pred_test)
f1_test = f1_score(y_test, y_pred_test)
confusion_matrix_test = confusion_matrix(y_test, y_pred_test)

print("Accuracy:", accuracy_test)
print("F1-score:", f1_test)
print("Confusion matrix:\n", confusion_matrix_test)

Accuracy: 0.9057383094851748
F1-score: 0.44386422976501305
Confusion matrix:
 [[5885  130]
 [ 509  255]]


# 7. CONCLUSIONES

Después de haber analizado diferentes modelos de clasificación podemos concluir:

- De los diferentes modelos analizados (Decision Tree, KNN, SVM, XGBoost, Random Forest, Logistic Regression y Naive Bayes), con el que obtenemos mejor rendimiento para este conjunto de datos es el XGBoost, con una precisión de 0.908085 en validación y de 0.909721 en prueba.

- En todos los modelos la métrica F1-Score muestra un rendimiento mucho más bajo para todos los modelos que la métrica que evalua la   precisión. 
    
    Cuando el F1-score muestra un rendimiento mucho más bajo que la precisión, puede indicar que el modelo está teniendo dificultades para recuperar correctamente los ejemplos positivos (baja recall), a pesar de tener una alta precisión en las predicciones positivas.

    Esto puede ocurrir cuando el conjunto de datos está desbalanceado y hay muchas más instancias de la clase negativa que de la clase positiva. En este caso, el modelo puede ser muy preciso en la clasificación de la clase mayoritaria (clase negativa), pero tiene dificultades para identificar correctamente los ejemplos de la clase minoritaria (clase positiva), lo que resulta en un bajo recall y, por lo tanto, un bajo F1-score.

- Una vez realizado el ensamble de los diferentes modelos vemos que no se puede apreciar una gran diferencia en el rendimiento según la precisión y el F1-score. 
  
  De todas formas, los ensembles pueden mitigar el sobreajuste, explorar de manera más exhaustiva el espacio de características y hiperparámetros, y mejorar la generalización del modelo, así que aunque no mejoren las métricas siempre son una buena opción.